# Pre-processing images

# Creating a csv file with labels and path

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import cv2
import random
import numpy as np
from tqdm import tqdm
import os


In [3]:
# get all the directory paths
dir_paths = os.listdir('ASL\\input\\asl_alphabet_train\\asl_alphabet_train')
dir_paths.sort()
root_path = 'ASL\\input\\asl_alphabet_train\\asl_alphabet_train'

# Add these parameters at the top
train_samples_per_class = 10   # 70% of total
val_samples_per_class = 100     # 15% of total
test_samples_per_class = 100    # 15% of total

# Modify the sampling logic
for idx, dir_path in tqdm(enumerate(dir_paths), total=len(dir_paths)):
    all_images = os.listdir(f"{root_path}/{dir_path}")
    random.shuffle(all_images)  # Shuffle the images first
    
    # Ensure we have enough images
    total_samples = train_samples_per_class + val_samples_per_class + test_samples_per_class
    if len(all_images) < total_samples:
        raise ValueError(f"Not enough images in {dir_path}. Need {total_samples}, have {len(all_images)}")
    
    # Randomly sample images for each dataset
    train_images = random.sample(all_images, train_samples_per_class)
    remaining_images = list(set(all_images) - set(train_images))

    val_images = random.sample(remaining_images, val_samples_per_class)
    test_images = random.sample(list(set(remaining_images) - set(val_images)), test_samples_per_class)
    
    # Process and save images for each dataset
    for dataset, images in zip(['train', 'val', 'test'], [train_images, val_images, test_images]):
        os.makedirs(f"ASL/input/preprocessed_image/{dataset}/{dir_path}", exist_ok=True)
        for image_name in images:
            image = cv2.imread(f"{root_path}/{dir_path}/{image_name}")
            image = cv2.resize(image, (224, 224))
            cv2.imwrite(f"ASL/input/preprocessed_image/{dataset}/{dir_path}/{image_name}", image)

100%|██████████| 29/29 [00:06<00:00,  4.22it/s]


In [4]:
import pandas as pd
import numpy as np
import os
import joblib
from sklearn.preprocessing import LabelBinarizer
from tqdm import tqdm
from imutils import paths

# Create separate DataFrames for each dataset
train_data = pd.DataFrame()
val_data = pd.DataFrame()
test_data = pd.DataFrame()

# Process each dataset separately
for dataset in ['train', 'val', 'test']:
    # Get image paths for the current dataset
    image_paths = list(paths.list_images(f'ASL\\input\\preprocessed_image\\{dataset}'))
    
    labels = []
    current_df = train_data if dataset == 'train' else val_data if dataset == 'val' else test_data
    
    for i, image_path in tqdm(enumerate(image_paths), total=len(image_paths)):
        label = image_path.split(os.path.sep)[-2]
        current_df.loc[i, 'image_path'] = image_path
        labels.append(label)
    
    # One hot encode the labels
    lb = LabelBinarizer()
    labels = lb.fit_transform(labels)
    
    # Add targets to the dataframe
    for i in range(len(labels)):
        index = np.argmax(labels[i])
        current_df.loc[i, 'target'] = int(index)
    
    # Shuffle and save the dataset
    current_df = current_df.sample(frac=1).reset_index(drop=True)
    current_df.to_csv(f'ASL/input/{dataset}_data.csv', index=False)
    
    # Save the binarized labels
    if dataset == 'train':  # Save only once using the train labels
        joblib.dump(lb, 'ASL/output/lb.pkl')

# Print sample data
print("Train data sample:")
print(train_data.head(5))
print("\nValidation data sample:")
print(val_data.head(5))
print("\nTest data sample:")
print(test_data.head(5))

# Print dataset shapes
print("Train data shape:", train_data.shape)
print("Validation data shape:", val_data.shape)
print("Test data shape:", test_data.shape)

100%|██████████| 2900/2900 [00:00<00:00, 3850.49it/s]


Train data sample:
                                       image_path  target
0  ASL\input\preprocessed_image\train\A\A1038.jpg     0.0
1  ASL\input\preprocessed_image\train\A\A1183.jpg     0.0
2  ASL\input\preprocessed_image\train\A\A1562.jpg     0.0
3  ASL\input\preprocessed_image\train\A\A1584.jpg     0.0
4  ASL\input\preprocessed_image\train\A\A1897.jpg     0.0

Validation data sample:
                                     image_path  target
0    ASL\input\preprocessed_image\val\A\A10.jpg     0.0
1  ASL\input\preprocessed_image\val\A\A1105.jpg     0.0
2  ASL\input\preprocessed_image\val\A\A1122.jpg     0.0
3  ASL\input\preprocessed_image\val\A\A1149.jpg     0.0
4  ASL\input\preprocessed_image\val\A\A1157.jpg     0.0

Test data sample:
                                      image_path  target
0  ASL\input\preprocessed_image\test\A\A1012.jpg     0.0
1  ASL\input\preprocessed_image\test\A\A1013.jpg     0.0
2  ASL\input\preprocessed_image\test\A\A1068.jpg     0.0
3  ASL\input\preprocessed

# Training the CNN model

In [5]:
import pandas as pd
import joblib
import numpy as np
import torch
import random
import albumentations
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms
import time
import cv2
import cnn_models
from tqdm import tqdm
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score
from torch.utils.data import Dataset, DataLoader

num_epoch = 200

# Seed everything
def seed_everything(SEED=42):
    random.seed(SEED)
    np.random.seed(SEED)
    torch.manual_seed(SEED)
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)
    torch.backends.cudnn.benchmark = True 

SEED = 42
seed_everything(SEED=SEED)

# Set computation device
# device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
device = "cpu"
print(f"Computation device: {device}")

# Dataset class
class ASLImageDataset(Dataset):
    def __init__(self, path, labels, augment=False):
        self.X = path
        self.y = labels
        self.augment = augment
        if self.augment:
            self.aug = albumentations.Compose([
                albumentations.Resize(224, 224),
            ])
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, i):
        image = cv2.imread(self.X[i])
        if image is None:
            raise ValueError(f"Failed to load image at {self.X[i]}")
            
        if self.augment:
            image = self.aug(image=np.array(image))['image']
        else:
            image = cv2.resize(image, (224, 224))
            
        image = np.transpose(image, (2, 0, 1)).astype(np.float32)
        label = self.y[i]
        return torch.tensor(image, dtype=torch.float), torch.tensor(label, dtype=torch.long)

# Read the data from the pre-split CSV files
train_df = pd.read_csv('ASL/input/train_data.csv')
val_df = pd.read_csv('ASL/input/val_data.csv')
test_df = pd.read_csv('ASL/input/test_data.csv')

# Get image paths and labels
xtrain = train_df.image_path.values
ytrain = train_df.target.values

xval = val_df.image_path.values
yval = val_df.target.values

xtest = test_df.image_path.values
ytest = test_df.target.values

# Create datasets
train_data = ASLImageDataset(xtrain, ytrain, augment=True)
val_data = ASLImageDataset(xval, yval, augment=False)
test_data = ASLImageDataset(xtest, ytest, augment=False)

batch_size = 1024  # set desired batch size

# Create dataloaders
trainloader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
valloader = DataLoader(val_data, batch_size=batch_size, shuffle=False)
testloader = DataLoader(test_data, batch_size=batch_size, shuffle=False)


Loading label binarizer...
Computation device: cpu


In [6]:
# Define the CNNTrainer class with only method definitions
class CNNTrainer:
    def __init__(self, model, device, criterion, optimizer, patience=5):
        self.model = model
        self.device = device
        self.criterion = criterion
        self.optimizer = optimizer
        self.patience = patience
        self.best_loss = float('inf')
        self.counter = 0
        
    def train_epoch(self, dataloader):
        self.model.train()
        running_loss = 0.0
        running_correct = 0
        all_preds = []
        all_probs = []
        all_targets = []
        
        for data, target in tqdm(dataloader, desc="Training"):
            data, target = data.to(self.device), target.to(self.device)
            self.optimizer.zero_grad()
            outputs = self.model(data)
            loss = self.criterion(outputs, target)
            loss.backward()
            self.optimizer.step()
            
            running_loss += loss.item()
            _, preds = torch.max(outputs.data, 1)
            running_correct += (preds == target).sum().item()
            
            all_preds.extend(preds.cpu().numpy())
            all_probs.extend(F.softmax(outputs, dim=1).cpu().detach().numpy())
            all_targets.extend(target.cpu().numpy())
            
        train_loss = running_loss / len(dataloader.dataset)
        train_accuracy = 100. * running_correct / len(dataloader.dataset)
        return train_loss, train_accuracy, all_preds, all_probs, all_targets

    def validate(self, dataloader):
        self.model.eval()
        running_loss = 0.0
        running_correct = 0
        all_preds = []
        all_probs = []
        all_targets = []
        
        with torch.no_grad():
            for data, target in tqdm(dataloader, desc="Validating"):
                data, target = data.to(self.device), target.to(self.device)
                outputs = self.model(data)
                loss = self.criterion(outputs, target)
                
                running_loss += loss.item()
                _, preds = torch.max(outputs.data, 1)
                running_correct += (preds == target).sum().item()
                
                all_preds.extend(preds.cpu().numpy())
                all_probs.extend(F.softmax(outputs, dim=1).cpu().detach().numpy())
                all_targets.extend(target.cpu().numpy())
                
        val_loss = running_loss / len(dataloader.dataset)
        val_accuracy = 100. * running_correct / len(dataloader.dataset)
        return val_loss, val_accuracy, all_preds, all_probs, all_targets
    
    def early_stopping(self, val_loss):
        improvement = self.best_loss - val_loss
        if improvement > 0.0001:  # Only consider it an improvement if loss decreases by more than 0.0001
            self.best_loss = val_loss
            self.counter = 0
            torch.save(self.model.state_dict(), 'ASL/output/best_model.pth')
            print(f"Model improved by {improvement:.6f}: best model saved.")
        else:
            self.counter += 1
            if self.counter >= self.patience:
                print(f"No significant improvement for {self.patience} epochs. Early stopping triggered.")
                return True
        return False

    def train(self, trainloader, valloader, num_epoch):
        train_loss, train_accuracy = [], []
        val_loss, val_accuracy = [], []  # Corrected line
        start = time.time()
        for epoch in range(num_epoch):
            print(f"\nEpoch {epoch+1} of {num_epoch}")
            
            # Training step
            epoch_train_loss, epoch_train_accuracy, train_preds, train_probs, train_targets = self.train_epoch(trainloader)
            train_loss.append(epoch_train_loss)
            train_accuracy.append(epoch_train_accuracy)
            
            # Calculate training metrics
            train_auc = roc_auc_score(train_targets, train_probs, multi_class='ovr')
            train_f1 = f1_score(train_targets, train_preds, average='weighted')
            
            # Validation step
            epoch_val_loss, epoch_val_accuracy, val_preds, val_probs, val_targets = self.validate(valloader)
            val_loss.append(epoch_val_loss)
            val_accuracy.append(epoch_val_accuracy)
            
            # Calculate validation metrics
            val_auc = roc_auc_score(val_targets, val_probs, multi_class='ovr')
            val_f1 = f1_score(val_targets, val_preds, average='weighted')
            
            # Print metrics
            print(f"Train - Loss: {epoch_train_loss:.4f}, Accuracy: {epoch_train_accuracy:.4f}, AUC: {train_auc:.4f}, F1: {train_f1:.4f}")
            print(f"Val   - Loss: {epoch_val_loss:.4f}, Accuracy: {epoch_val_accuracy:.4f}, AUC: {val_auc:.4f}, F1: {val_f1:.4f}")
            
            # Early stopping check
            if self.early_stopping(epoch_val_loss):
                break
                
        end = time.time()
        print(f"\nTraining complete in {(end - start)/60:.2f} minutes")
        train_time = (end - start)/60
        
        # Load and return the best model
        best_model = self.model
        best_model.load_state_dict(torch.load('ASL/output/best_model.pth'))
        return best_model, train_loss, train_accuracy, val_loss, val_accuracy, train_time

    def evaluate(self, dataloader):
        self.model.eval()
        running_correct = 0
        all_preds = []
        all_probs = []
        all_targets = []
        total_time = 0.0
        num_samples = 0
        
        with torch.no_grad():
            for data, target in tqdm(dataloader, desc="Testing"):
                data, target = data.to(self.device), target.to(self.device)
                
                # Measure inference time
                start_time = time.time()
                outputs = self.model(data)
                end_time = time.time()
                
                total_time += (end_time - start_time)
                num_samples += data.size(0)
                
                _, preds = torch.max(outputs.data, 1)
                running_correct += (preds == target).sum().item()
                all_preds.extend(preds.cpu().numpy())
                all_probs.extend(F.softmax(outputs, dim=1).cpu().detach().numpy())
                all_targets.extend(target.cpu().numpy())
        
        # Calculate metrics
        accuracy = 100. * running_correct/len(dataloader.dataset)
        test_auc = roc_auc_score(all_targets, all_probs, multi_class='ovr')
        test_f1 = f1_score(all_targets, all_preds, average='weighted')
        fps = num_samples / total_time
        
        return accuracy, test_auc, test_f1, fps
            


In [7]:
from cnn_models import CustomCNN
from resnet_models import ResnetModel

# Initialize models
models_dict = {
    'ResNet': ResnetModel().to(device),
    'CNN': CustomCNN().to(device)
}

# Train each model
results = {}
for model_name, model in models_dict.items():
    print(f"\nTraining {model_name} model...")
    
    # Initialize optimizer and criterion
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss()
    
    # Create trainer
    trainer = CNNTrainer(model, device, criterion, optimizer, patience=5)
    
    # Train the model
    best_model, train_loss, train_accuracy, val_loss, val_accuracy, train_time = trainer.train(trainloader, valloader, num_epoch)
    
    # Evaluate on test set
    test_accuracy, test_auc, test_f1, fps = trainer.evaluate(testloader)
    
    # Store results
    results[model_name] = {
        'model': best_model,
        'train_loss': train_loss,
        'train_accuracy': train_accuracy,
        'val_loss': val_loss,
        'val_accuracy': val_accuracy,
        'test_accuracy': test_accuracy,
        'test_auc': test_auc,
        'test_f1': test_f1,
        'fps': fps,
        'train_time': train_time
    }
    
    # Save the best model
    torch.save(best_model.state_dict(), f'ASL/output/best_{model_name.lower()}_model.pth')
    print(f"Best {model_name} model saved")




Loading label binarizer...

Training ResNet model...

Epoch 1 of 200


Validating: 100%|██████████| 3/3 [00:37<00:00, 12.63s/it]


Train - Loss: 0.0123, Accuracy: 2.0690, AUC: 0.4589, F1: 0.0094
Val   - Loss: 0.0038, Accuracy: 2.0690, AUC: 0.4979, F1: 0.0038
Model improved by inf: best model saved.

Epoch 2 of 200


Validating: 100%|██████████| 3/3 [00:36<00:00, 12.14s/it]


Train - Loss: 0.0118, Accuracy: 3.7931, AUC: 0.5154, F1: 0.0356
Val   - Loss: 0.0037, Accuracy: 3.3793, AUC: 0.5004, F1: 0.0101
Model improved by 0.000117: best model saved.

Epoch 3 of 200


...

Train - Loss: 0.0116, Accuracy: 6.8966, AUC: 0.6122, F1: 0.0188
Val   - Loss: 0.0035, Accuracy: 3.9310, AUC: 0.5668, F1: 0.0043

Epoch 6 of 200


Validating: 100%|██████████| 3/3 [00:22<00:00,  7.36s/it]
C:\Users\sovte\AppData\Local\Temp\ipykernel_34144\1473212738.py:119: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  

Train - Loss: 0.0116, Accuracy: 3.4483, AUC: 0.6346, F1: 0.0030
Val   - Loss: 0.0035, Accuracy: 3.4483, AUC: 0.5677, F1: 0.0024
No significant improvement for 5 epochs. Early stopping triggered.

Training complete in 2.58 minutes


Testing: 100%|██████████| 3/3 [00:21<00:00,  7.27s/it]

Best CNN model saved


In [12]:
# Convert results to DataFrame and save as Excel
results_df = pd.DataFrame.from_dict(results, orient='index')
results_df = results_df.reset_index().rename(columns={'index': 'Model'})

# Save to Excel
results_df.to_excel('ASL/output/training_results.xlsx', index=False)
print("Training results saved to ASL/output/training_results.xlsx")

Training results saved to ASL/output/training_results.xlsx


# Testing the Model

In [9]:
# # Process all images
# fig, axes = plt.subplots((len(all_files) + 4) // 5, 5, figsize=(20, 4*((len(all_files) + 4) // 5)))
# axes = axes.flatten()

# for i, img_file in enumerate(all_files):
#     # Load and process image
#     test_img = os.path.join(test_dir, img_file)
#     image = cv2.imread(test_img)
#     image_copy = image.copy()
    
#     # Preprocess image
#     image = aug(image=np.array(image))['image']
#     image = np.transpose(image, (2, 0, 1)).astype(np.float32)
#     image = torch.tensor(image, dtype=torch.float)
#     image = image.unsqueeze(0)
    
#     # Make prediction
#     start = time.time()
#     outputs = model(image)
#     _, preds = torch.max(outputs.data, 1)
#     end = time.time()
    
#     # Add prediction to image
#     cv2.putText(image_copy, lb.classes_[preds], (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)
    
#     # Display in subplot
#     axes[i].imshow(cv2.cvtColor(image_copy, cv2.COLOR_BGR2RGB))
#     axes[i].axis('off')
#     axes[i].set_title(f"{lb.classes_[preds]}\n({(end-start):.3f}s)")

# # Hide any empty subplots
# for j in range(i+1, len(axes)):
#     axes[j].axis('off')

# plt.tight_layout()
# plt.show()

# Real time capture